# Démonstration 10

## 1. Algorithmes de points intérieurs

In [10]:
using LinearAlgebra
using JuMP, Clp

┌ Info: Precompiling Clp [e2554f3b-3117-50c0-817c-e040a3ddf72d]
└ @ Base loading.jl:1278


In [11]:
# Klee-Minty problem

n = 3

m = Model(with_optimizer(Clp.Optimizer))

@variable(m, x[1:n] >= 0)
for i in 1:n
   @constraint(m, 2*sum(10^(i-j)*x[j] for j = 1:i-1)+x[i] <= 100^(i-1))
end
@objective(m, Max, sum(10^(n-j)*x[j] for j = 1:n))

print(m)

Max 100 x[1] + 10 x[2] + x[3]
Subject to
 x[1] <= 1.0
 20 x[1] + x[2] <= 100.0
 200 x[1] + 20 x[2] + x[3] <= 10000.0
 x[1] >= 0.0
 x[2] >= 0.0
 x[3] >= 0.0


In [12]:
status = optimize!(m)

Coin0506I Presolve 0 (-3) rows, 0 (-3) columns and 0 (-6) elements
Clp3002W Empty problem - 0 rows, 0 columns and 0 elements
Clp0000I Optimal - objective value 10000
Coin0511I After Postsolve, objective 10000, infeasibilities - dual 0 (0), primal 0 (0)
Clp0032I Optimal objective 10000 - 0 iterations time 0.002, Presolve 0.00


In [13]:
value.(x)

3-element Array{Float64,1}:
     0.0
     0.0
 10000.0

In [14]:
c = [[-10^(n-i) for i = 1:n]; [0 for i = 1:n]]
b = [100^(i-1) for i = 1:n]
A = [1 [0*i for i in 1:n-1]']
for i = 2:n
    A = [A; [[2*10^(i-j) for j in 1:i-1]' 1 [0 for i = 1:n-i]']]
end
B = A
A = [A I]
x = [0.5]
for i = 2:n
    x = [ x ; (0.5*b[i]-dot(A[i,1:i-1],x[1:i-1]))]
end
x = [ x; b-B*x ]
A*x

3-element Array{Float64,1}:
     1.0
   100.0
 10000.0

In [15]:
m, n = size(A)
y = [-1 for i = 1:m]
s = c-A'y

6-element Array{Int64,1}:
 121
  11
   0
   1
   1
   1

In [16]:
x

6-element Array{Float64,1}:
    0.5
   40.0
 4100.0
    0.5
   50.0
 5000.0

In [17]:
y

3-element Array{Int64,1}:
 -1
 -1
 -1

In [18]:
## Adapted from https://github.com/chkwon/jpor_codes/blob/master/chap7/primal_path_following.jl

function primal_path_following(c,A,b,x0;
            mu=0.9, alpha=0.9, epsilon=1e-9, max_iter=1000)
  # Initialization
  x = x0
  n = length(x)
  m = length(b)
  e = ones(n,1)

  # Preparing variables for the trajectories
  trace = copy(x)
    
  for i=1:max_iter
    # Computing
    invX = Diagonal(1 ./ x)
    mu = alpha * mu

    # Solving the linear system
    LHS = [ mu*invX^2       -A'     ;
               A          zeros(m,m)  ]
    RHS = [ mu*invX*e - c ;
            zeros(m,1) ]
    sol = LHS \ RHS

    # Update
    d = sol[1:n]
    p = sol[n+1:end]
    x = x + d
    s = c - A'*p

    # Recording the trajectories of x1 and x2
    trace = [trace x]

    # Optimality check
    if dot(x,s) < epsilon
      break
    end
  end

  return trace
end

primal_path_following (generic function with 1 method)

In [19]:
x0 = x
trajectory = primal_path_following(c, A, b, x0)

6×2 Array{Float64,2}:
    0.5      1.09411
   40.0   2373.52
 4100.0      1.23949e7
    0.5     -0.0941119
   50.0  -2295.4
 5000.0     -1.24326e7

L'implantation du chemin primale échoue comme le pas est trop long, de sorte que les contraintes de non-négativité ne sont pas satisfaites.

In [20]:
function primaldual_path_following(c,A,b,x,y,s, μ = 100, α = 0.1, γ=0.1, epsilon=1e-9, max_iter=1000)

    for ν = 1:10

        iter = max_iter

        for k = 1:max_iter
            
            X = diagm(x)
            S = diagm(s)
        H = [ zeros(n,n) A' I ;
              A zeros(m,m) zeros(m,n);
              S zeros(n,m) X ]
        t = [ zeros(n) ; zeros(m); μ*ones(n)-X*S*ones(n) ]
        Δ = H\t
        α = 0.01
        new = [x; y; s]+α*Δ
        x = new[1:n]
        y = new[n+1:n+m]
        s = new[n+m+1:2*n+m]
        if (norm(x.*s-μ*ones(n)) <= 1/10^(ν/2))
            iter = k
            break
        end

        end

        println(ν, ". ", iter, " ", x.*s)
        μ = γ*μ

    end
    
    return x
end

primaldual_path_following (generic function with 6 methods)

In [21]:
primaldual_path_following(c,A,b,x0,y,s)

1. 961 [99.99746053993127, 100.02171400967336, 99.993648579146, 99.99365835390805, 99.99680974204617, 100.31302024100387]
2. 767 [10.040595161153496, 10.040459669709007, 10.040389675214719, 10.040238718109789, 10.040340237760457, 10.040549468526802]
3. 652 [1.0129063168319479, 1.0128939366376868, 1.0128918505520559, 1.0128808919398564, 1.0128890311747112, 1.012892753552498]
4. 539 [0.1040536340599087, 0.10405317170454113, 0.1040530992214235, 0.1040526979588318, 0.1040529942735418, 0.10405312545627189]
5. 427 [0.011287089171121192, 0.011287069583964754, 0.011287066541942616, 0.011287049596399033, 0.011287062092694734, 0.011287067624233314]
6. 322 [0.0014044135457085846, 0.0014044123316333727, 0.001404412143414101, 0.001404411093730764, 0.0014044118675834777, 0.001404412210182854]
7. 231 [0.00022798094572352786, 0.00022798080987718126, 0.00022798078882489678, 0.0002279806713888193, 0.00022798075796011253, 0.00022798079628831363]
8. 167 [5.069078232691281e-5, 5.069075616673979e-5, 5.06907

6-element Array{Float64,1}:
    4.1588989932890645e-8
    4.1588971672448483e-7
 9999.999979205506
    0.9999999584110095
   99.99999875232997
    4.158896984580097e-6

On converge, mais c'est extrêmement lent!

## Question 2

Nous considérons un problème de programmation linéaire sous forme standard et
nous supposons que les ensemble réalisables du primal et du dual ont des
intérieurs non vides. Montrer que si l'ensemble réalisable du primal est borné,
alors l'ensemble réalisable du dual ne peut pas être borné. Est-il possible
d'avoir des ensembles réalisables non bornés pour le primal et le dual?

Soit $\left(x(\mu), y(\mu), s(\mu) \right)$, $0<\mu<\infty$ Si l'ensemble réalisable de primal
est borné, $x(\mu)$ converge vers le centre analytique de l'ensemble réalisable
du primal. Dès lors, $x(\mu)$ tend vers un certain $x^{\star} $ pour $\mu \to
\infty$, avec $x^{\star} $ fini.  D'autre part, on sait que

\begin{align*}
c^{t} x(\mu) - b^{t} y(\mu) = \eta \mu
\end{align*}
Dès lors, pour $\mu \to \infty$, 
\begin{align*}
\lim_{\mu\to \infty} b^t y(\mu) = c^t x^{\star} - \infty = - \infty
\end{align*}
Dès lors, $y(\mu)$ ne peut pas converger vers un certain $y^{\star} $ fini comme $\mu$ tend vers l'infini. 
Or ce serait le cas si l'ensemble dual était borné.

On peut avoir des ensembles non bornés pour le primal et le dual. Considérions la paire
\begin{align*}
&\min\ x_1 + x_2 \\
&\begin{aligned}
\text{s.a.} &\\
&  x_1 = 1 ,\\
& x_1, x_2 \ge 0.
\end{aligned}
\end{align*}

\begin{align*}
&\max\ \lambda \\
&\begin{aligned}
\text{s.a.} &\\
& \lambda \le 1 ,\\
\end{aligned}
\end{align*}
Les deux ensemble ont des intérieurs non vides et ne sont pas bornés.

# Question 3

Considérons le polyhèdre $P = \{ x \,|\, Ax \leq b \}$. Supposons que $P$ est borné et que son intérieur est non-vide, ou en d'autres termes, qu'il existe $x$ tel que $Ax < b$.

- Écrivez la fonction potentiel à minimiser pour trouver le centre analytique de $P$.
- En annulant le gradient de la fonction potentiel, écrivez le système non-linéaire à résoudre pour trouver le centre analytique.
- Le système obtenu au point précédent peut être résolu à l'aide de la méthode de Newton. Écrivez une itération de la résolution du système pas la méthode de Newton.

Notons $a_i^T$ la $i^e$ ligne de $A$. La contrainte $Ax \leq b$ peut se réécrire
$$
b - a_i^Tx \geq 0,\ i = 1,\ldots,m,
$$
en supposant que $A \in R^{m \times n}$.

La fonction potentiel à minimiser s'écrit dès lors
$$
\Psi(x) = -\sum_{i = 1}^m \log(b_i-a_i^Tx)
$$
Son gradient s'écrit
$$
\nabla_x \Psi(x) =
\begin{pmatrix}
 \sum_{i = 1}^m \frac{a_{i1}}{b_i-a_i^Tx} \\
 \sum_{i = 1}^m \frac{a_{i2}}{b_i-a_i^Tx} \\
\vdots \\
 \sum_{i = 1}^m \frac{a_{in}}{b_i-a_i^Tx}
\end{pmatrix}
=
 \sum_{i = 1}^m \frac{a_{i}}{b_i-a_i^Tx} \\
= A^T\begin{pmatrix}
\frac{1}{b-a_1x} \\
\frac{1}{b-a_2x} \\
\vdots \\
\frac{1}{b-a_mx}
\end{pmatrix}
$$
que nous cherchons à annuler.
En posant
$$
s_i = b_i - a_i^Tx, \quad y_i = \frac{1}{s_i},\ i = 1,\ldots,m,
$$
le système peut se réécrire comme
$$
A^Ty = 0,\ \textrm{avec}\ y = \begin{pmatrix} y_1 \\ \vdots \\ y_m \end{pmatrix}.
$$

Pour obtenir l'itération de Newton, nous devons tout d'abord écrire la matrice hessienne de la fonction potentiel:
$$
\nabla^2_{xx} \Psi(x) =
\sum_{i = 1}^m \frac{a_{i}a_i^T}{(b_i-a_i^Tx)^2} = A^TS^{-2}A = A^TY^2A,
$$
Une itération de Newton consiste à prendre
$$
x^+ = x - (\nabla^2_{xx} \Psi(x))^{-1}\nabla_x \Psi(x) = x - (A^TY^2A)^{-1}A^Ty.
$$

# Implémentation de la Méthode de Mehrotra

Arnaud L'Heureux, Décembre 2019

Étant donné la paire primale-duale d'un problème de programmation linéaire : 
$$\min_x c^Tx$$
$$\text{s.à.}\ \ Ax = b$$
$$x \geq 0$$
et
$$\max_{\lambda,s} b^T\lambda$$
$$\text{s.à.}\ \ A^T\lambda + s = c$$
$$s \geq 0$$
Nous avons les conditons KKT suivantes qui doivent être satisfaites pour que la solution au problème soit optimale:
$$Ax = b$$
$$A^T\lambda + s = c$$
$$XSe = 0$$
$$(x,s) \geq 0$$
où $X$ et $S$ sont des matrices diagonales ayant $x$ et $s$ comme diagonales. La condition $Ax = b$, nous indique que la solution est réalisable pour le primal alors que $A^T\lambda + s = c$ implique que le dual est réalisable. Tout point réalisable pour le primal ainsi que le dual satisfaisant la condition $XSe = 0$ implique que ce point est optimal pour le primal et le dual.

In [ ]:
using LinearAlgebra

# Heuristique proposée par Mehrotra
function genStart(A:: Matrix, b:: Vector, c:: Vector)
    x̅ = A'*((A*A')\b)
    λ̅ = (A*A')\A*c
    s̅ = c - A'*λ̅

    δx = max(-1.5*minimum(x̅),0)
    δs = max(-1.5*minimum(s̅),0)

    # δx et δs sont des scalaires, et doivent être ajoutés à chaque composante, c'est pourquoi nous utilisons .+
    num = dot(x̅ .+ δx, s̅ .+ δs)
    δx̅ = δx + num/(2*sum(s̅ .+ δs))
    δs̅ = δs + num/(2*sum(x̅ .+ δx))

    return (x̅ .+ δx̅, λ̅ , s̅ .+ δs̅)
end

L'algorithme de Mehrotra suit.

In [ ]:
using SparseArrays

function Mehrotra(A:: Matrix, b:: Vector, c:: Vector,
                  ε:: Float64, maxIter:: Int64 = 1000, sparseMode:: Bool = True)

    (m,n) = size(A)  # calcul la dimension de A
    
    if (sparseMode)
        temp = sparse(A)
        density = nnz(temp)/(m*n)
       
        # Si la densité de A est inférieure à 20%, nous travaillons avec une matrice creuse
        if (density < .2)
            A = sparse(A)
            b = sparse(b)
            c = sparse(c)
        end
    end

    # Calcul du point de départ
    res = genStart(A,b,c)
    x = res[1]
    λ = res[2]
    s = res[3]
    
    iter = 0

    # Nous utilisons le critère d'arrêt par défaut dans le test de boucle.
    # Le test de convergence se fera en interne, et nous pourrons alors quitter la fonction.
    while (iter < maxIter)
        iter += 1
        
        # Calcul des résidus
        rp = A*x-b
        rd = A'*λ + s - c
        rc = x .* s

        # μ est aligné sur le saut de dualité
        μ = dot(x,s)/n
        
        # Nous testons la qualité de la solution
        if (max(μ, norm(rd), norm(rp)) < ε)
            return (x,λ,s)
        end
        
        # Calcul de M = AXS^-1A' et du factory de Cholesky R de M (M=R'R)
        M = A*diagm(x./s)*A'
        R = cholesky(M).U

        # Calcul des termes de droite
        td = rp - A*((rc - x.*rd)./s)
        
        # Étape prédictrice
        Δλ_p = R\(R'\td)
        Δs_p = rd - A'*Δλ_p
        Δx_p = (rc - x.*Δs_p)./s
 
        # Calculs des longueur de pas potentiels les plus longs
        # Pas primal
        α_p = 1/maximum([1 ; Δx_p ./ x])
        # Pas dual
        α_d = 1/maximum([1 ; Δs_p ./ s])
        
        # Paramètre centralisant
        σ = (dot(x - α_p*Δx_p, s - α_d*Δs_p)/(n*μ))^3
        
        # Étape correctrice
        rc = rc .- σ*μ + Δx_p.*Δs_p
        td = rp - A*((rc - x.*rd)./s)
        
        Δλ_c = R\(R'\td)
        Δs_c = rd - A'*Δλ_c
        Δx_c = (rc - x.*Δs_c)./s
        
        η = max(0.995, 1-μ)
        
        # Pas primal
        α_p = η/maximum([η ; Δx_c./x])
        # Pas dual
        α_d = η/maximum([η ; Δs_c./s])
        
        # Mise à jour da la solution
        x -= α_p*Δx_c
        λ -= α_d*Δλ_c
        s -= α_d*Δs_c
    end
end

### Exemple

Soit le problème:
\begin{align*}
\min z\ &= 5x_1 + 2x_2 -4x_3 \\
\text{s.à. } & 6x_1 + x_2 - 2x_3 - x_4 = 5 \\
& x_1 + x_2 + x_3 + x_5 = 4 \\
& 6x_1 + 4x_2 -2x_3 - x_6= 10 \\
& x_i \geq 0, \ i = 1,\ldots,6.
\end{align*}

In [ ]:
A = [6 1 -2 -1 0 0; 1 1 1 0 1 0; 6 4 -2 0 0 -1]
b = [5; 4; 10]
c = [5; 2; -4; 0; 0; 0]

Nous commençons par trouver un point de départ $(x_0, \lambda_0, s_0)$.

In [ ]:
start = genStart(A,b,c)

println(A*start[1]-b)
println(A'*start[2]-c)
println(start[1], " ", start[3])

Nous voyons que seules les contraintes de non-négativité sont satisfaite, mais ni $x_0$, ni $\lambda_0$, ne sont réalisables.

Appliquons l'algorithe de Mehrotra.

In [ ]:
x = Mehrotra(A,b,c,1e-10,10,true)[1]

println("x = ", x)
println("Ax = ", A*x)
println("z = c^Tx = ", dot(c,x))

Nous obtenons la solution:

$$x = \begin{pmatrix}
  0.9999999999999999 \\
  1.6666666666666656 \\
  1.3333333333333355 \\
  3.5867130386728067 \times 10^{-19}\\
  1.5140739781564760 \times 10^{-19}\\
  3.8753111845137296 \times 10^{-19}
 \end{pmatrix}  \approx \begin{pmatrix}
  1 \\
  \frac{5}{3} \\
  \frac{4}{3} \\
  0\\
  0\\
  0
 \end{pmatrix}
$$
La solution est
$$x_1 = 1, \ \ x_2 = \frac{5}{3}, \ \ x_3 = \frac{4}{3}, \ \ x_4 = x_5 = x_6 = 0,$$
qui est bien réalisable puisque $Ax = b$. Cette solution nous donne la valeur de la fonction objectif $z = 3$.